In [49]:
from astropy.table import Table, join
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
import statistics as st

#ForGroup = Table.read('../../GAMA_DATA/G3CFoFGroupv10.fits')
#print(f"ForGroup : {len(ForGroup)}")
## Clean Data

GroupGal = Table.read('../../GAMA_DATA/G3CGalv10.fits')
# Clean data
GroupGal = GroupGal[GroupGal['GroupID'] > 0 ]
print(f"GroupGal : {len(GroupGal)}")

StellarMasses = Table.read('../../GAMA_DATA/StellarMassesv19.fits')
# Clean Data
#StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['metal'] > 0.001]


StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['uminusr'] < 5]
StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['metal'] > 0.01]
StellarMasses = StellarMasses['CATAID','logmstar']
print(f"StellarMasses : {len(StellarMasses)}")

envMeasures = Table.read('../../GAMA_DATA/EnvironmentMeasuresv05.fits')
print(f"EnvMeasures : {len(envMeasures)}")
# Clean Data
envMeasures = envMeasures[envMeasures['SurfaceDensity'] < 50]
envMeasures = envMeasures[envMeasures['AGEDenParFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCylFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCyl'] > 0]
print(f"EnvMeasures : {len(envMeasures)}")

#envClass = Table.read('../../GAMA_DATA/GalaxiesClassifiedv01.fits')
# Clean Data

visualMorph = Table.read('../../GAMA_Data/VisualMorphologyv03.fits')
print(f"visualMorph : {len(visualMorph)}")
print(visualMorph.colnames)
## Clean Data
#visualMorphElliptical = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 1]
#visualMorphSpiral = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 10]
visualMorph = visualMorph[visualMorph['ELLIPTICAL_CODE'] != 0]
visualMorph = visualMorph[visualMorph['ELLIPTICAL_CODE'] < 50]

GroupEnvs = join(GroupGal,envMeasures,keys='CATAID',join_type='inner')
print(GroupEnvs.colnames)

cols = ['GroupID','ELLIPTICAL_CODE','CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar']
DLG_Groups = GroupEnvs.group_by('GroupID')
GroupMeanEnvs = DLG_Groups['GroupID','CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'].groups.aggregate(lambda x: np.mean(x))
print("GroupMeanEnvs")
print(GroupMeanEnvs.colnames)

GroupVis = join(GroupGal,visualMorph,keys='CATAID',join_type='inner')

dfV  = GroupVis.to_pandas()
dfV.set_index('GroupID')
dfM = GroupMeanEnvs.to_pandas()
dfM.set_index('GroupID')
print(dfV.columns)
print(dfM.columns)
df = pd.merge(dfV, dfM, on=["GroupID", "GroupID"])
#print(df)
#GalCounts = df.groupby(['GroupID','ELLIPTICAL_CODE']).size().reset_index(name='Counts')
#GalCounts = df.groupby(['GroupID','ELLIPTICAL_CODE','CountInCyl','DistanceTo5nn']).size().reset_index(name='Counts')
GalCounts = df.groupby(cols).size().reset_index(name='Counts')
print('GalCounts')
print(GalCounts)
#df.set_index('GroupID')

#SumECodes = GalCounts.groupby(['GroupID', 'ELLIPTICAL_CODE']).agg({'Counts': 'sum'})
SumECodes = GalCounts.groupby(cols).agg({'Counts': 'sum'})
print('SumECodes')
print(SumECodes)
Range = GalCounts.groupby(['GroupID']).agg({'Counts': 'sum'})
#Range = GalCounts.groupby(cols).agg({'Counts': 'sum'})
print('Range')
print(Range)
Percent=SumECodes.div(Range, level='GroupID') * 100
print('Count is percentage')
print(Percent)
#df1=pd.merge(Percent, dfM, on=["GroupID", "GroupID"])
#print(df1)
#df2=pd.merge(df1,dfV, on=["GroupID", "GroupID"])
#print(df2)


alphaVal = .3
binCount = 100
 

GroupGal : 83093
StellarMasses : 114662
EnvMeasures : 67996
EnvMeasures : 40574
visualMorph : 38795
['CATAID', 'ELLIPTICAL', 'ELLIPTICAL_CODE', 'HUBBLE_TYPE_SPD', 'HUBBLE_TYPE_SPD_CODE', 'HUBBLE_TYPE_LSK', 'HUBBLE_TYPE_LSK_CODE', 'HUBBLE_TYPE_ASGR', 'HUBBLE_TYPE_ASGR_CODE', 'HUBBLE_TYPE', 'HUBBLE_TYPE_CODE', 'ASGR_DISTURBED', 'ASGR_NORMAL', 'ASGR_UNSURE', 'ASGR_CLASSIFIERS', 'ASGR_PROBDIST', 'P_EL', 'P_CS', 'P_EL_DEBIASED', 'P_CS_DEBIASED', 'URL_IMG']


['CATAID', 'RA_1', 'Dec', 'Z', 'Rpetro', 'DM_100_25_75', 'SURVEY_CODE', 'SigErr', 'GroupID', 'RankIterCen', 'SepIterCen', 'CoSepIterCen', 'AngSepIterCen', 'RankBCG', 'SepBCG', 'CoSepBCG', 'AngSepBCG', 'RankCen', 'SepCen', 'CoSepCen', 'AngSepCen', 'RA_2', 'DEC', 'Z_TONRY', 'nQ', 'M_r', 'DistanceToEdge', 'Completeness', 'DistanceTo5nn', 'SurfaceDensity', 'SurfaceDensityErr', 'SurfaceDensityFlag', 'CountInCyl', 'CountInCylErr', 'CountInCylFlag', 'AGEDenPar', 'AGEErr', 'AGEScale', 'AGEDenParFlag']
GroupMeanEnvs
['GroupID', 'CountInCyl', 'DistanceTo5nn', 'SurfaceDensity', 'AGEDenPar']
Index(['CATAID', 'RA', 'Dec', 'Z', 'Rpetro', 'DM_100_25_75', 'SURVEY_CODE',
       'SigErr', 'GroupID', 'RankIterCen', 'SepIterCen', 'CoSepIterCen',
       'AngSepIterCen', 'RankBCG', 'SepBCG', 'CoSepBCG', 'AngSepBCG',
       'RankCen', 'SepCen', 'CoSepCen', 'AngSepCen', 'ELLIPTICAL',
       'ELLIPTICAL_CODE', 'HUBBLE_TYPE_SPD', 'HUBBLE_TYPE_SPD_CODE',
       'HUBBLE_TYPE_LSK', 'HUBBLE_TYPE_LSK_CODE', 'HUBBLE_